In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.classifier import StackingClassifier
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve, auc

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
df = pd.read_csv('bank-full.csv',sep=';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [19]:
df.tail()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no
45210,37,entrepreneur,married,secondary,no,2971,no,no,cellular,17,nov,361,2,188,11,other,no


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [21]:
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [22]:
df.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [23]:
le = LabelEncoder()
for i in df.select_dtypes('object').columns:
    df[i] = le.fit_transform(df[i]) # doing this to change the object datatype into numerical datatype
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3,0
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3,0
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3,0
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3,0
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3,0


In [24]:
X=df.drop('y',axis=1)
y=df['y']

In [25]:
mms=MinMaxScaler()
X[['age','balance','day','duration']] = mms.fit_transform(X[['age','balance','day','duration']])
X[['age','balance','day','duration']].head() # balance,duration,age and day have wide ranges when compared to others

,age,balance,day,duration
0,0.519481,0.092259,0.133333,0.053070
1,0.337662,0.073067,0.133333,0.030704
2,0.194805,0.072822,0.133333,0.015453
3,0.376623,0.086476,0.133333,0.018707
4,0.194805,0.072812,0.133333,0.040260


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.25, random_state=0)
print("X-Train:",X_train.shape,"\nX-Test:",X_test.shape) # train-test split of the data

X-Train: (33908, 16) 
X-Test: (11303, 16)


In [27]:
# Models used for ensembling

#base models
models = {
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'XGBoost': XGBClassifier(),
    'KNN_Bagging': BaggingClassifier(base_estimator=KNeighborsClassifier(), n_estimators=10, random_state=42),
    'LogReg_Bagging': BaggingClassifier(base_estimator=LogisticRegression(solver='liblinear'), n_estimators=10, random_state=42),
    'Stacking': StackingClassifier(classifiers=[
        RandomForestClassifier(),
        GradientBoostingClassifier(),
        AdaBoostClassifier(),
        XGBClassifier()
    ],
    meta_classifier=LogisticRegression(solver='liblinear'),
    use_probas=True, average_probas=False),
    'Voting': VotingClassifier(estimators=[
        ('RandomForest', RandomForestClassifier()),
        ('GradientBoosting', GradientBoostingClassifier()),
        ('AdaBoost', AdaBoostClassifier()),
        ('XGBoost', XGBClassifier())
    ])
}


In [28]:
#hyperparameters list
ranges = {
    'RandomForest': {
        'n_estimators': range(10, 201),
        'max_depth': [None] + list(range(10, 31)),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 9),
        'bootstrap': [True, False]
    },
    'GradientBoosting': {
        'n_estimators': range(10, 201),
        'learning_rate': [0.01, 0.1, 0.2, 0.3],
        'max_depth': [3, 5, 7, 10],
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 9)
    },
    'AdaBoost': {
        'n_estimators': range(10, 201),
        'learning_rate': [0.01, 0.1, 0.2, 0.3]
    },
    'XGBoost': {
        'n_estimators': range(10, 201),
        'learning_rate': [0.01, 0.1, 0.2, 0.3],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': range(1, 9),
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    },
    'KNN_Bagging': {
        'base_estimator__n_neighbors': range(3, 21),
        'base_estimator__weights': ['uniform', 'distance'],
        'base_estimator__p': [1, 2],
        'n_estimators': range(5, 16),
        'max_samples': [0.8, 1.0],
        'max_features': [0.8, 1.0]
    },
    'LogReg_Bagging': {
        'base_estimator__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'base_estimator__penalty': ['l1', 'l2'],
        'n_estimators': range(5, 16),
        'max_samples': [0.8, 1.0],
        'max_features': [0.8, 1.0]
    },
    'Stacking': {
        'meta_classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'use_probas': [True],
        'average_probas': [False]
    },
    'Voting': {
        'voting': ['hard', 'soft']
    }
}

In [33]:
results_list = []

# Iterate over models
for model_name, model in models.items():
    # Initialize RandomizedSearchCV
    random_search = RandomizedSearchCV(
        model,
        param_distributions=ranges[model_name],
        n_iter=10,
        scoring='roc_auc',
        n_jobs=-1,
        cv=StratifiedKFold(n_splits=3),
        random_state=0,
        verbose=2
    )

    # Perform RandomizedSearchCV on the training data
    random_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = random_search.best_params_

    # Create a new instance of the model with the best hyperparameters
    best_model = model.set_params(**best_params)

    # Fit the best model on the training data
    best_model.fit(X_train, y_train)

    # Make predictions on the training and test sets
    y_train_pred = best_model.predict(X_train)
    y_test_pred = best_model.predict(X_test)

    # Calculate AUC scores
    auc_train = roc_auc_score(y_train, best_model.predict_proba(X_train)[:, 1])
    auc_test = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])

    # Calculate F1 scores
    f1_train = f1_score(y_train, y_train_pred)
    f1_test = f1_score(y_test, y_test_pred)

    print('Model In Use:', model_name,
          'AUC_train:', auc_train, 'AUC_test:', auc_test,
          'F1_train:', f1_train, 'F1_test:', f1_test)

    results_list.append({
        'Model': model_name,
        'AUC_train': auc_train,
        'AUC_test': auc_test,
        'F1_train': f1_train,
        'F1_test': f1_test
    })

results_df = pd.DataFrame(results_list)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Model In Use: RandomForest AUC_train: 0.9858306814496376 AUC_test: 0.9258052999618028 F1_train: 0.7741840635107322 F1_test: 0.5063636363636365
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Model In Use: GradientBoosting AUC_train: 0.9528416633349626 AUC_test: 0.9279607967717658 F1_train: 0.6383103549427985 F1_test: 0.5321814254859611
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Model In Use: AdaBoost AUC_train: 0.9146744036564183 AUC_test: 0.9038830332391639 F1_train: 0.43977310643977313 F1_test: 0.43499511241446726
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Model In Use: XGBoost AUC_train: 0.9665671328491287 AUC_test: 0.929573136371716 F1_train: 0.6906022028322129 F1_test: 0.5423291792006877
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Model In Use: KNN_Bagging AUC_train: 0.9999888035196814 AUC_test: 0.8552951587832855 F1_train: 0.9804577012085368 F1_test: 0.24

,Model,AUC_train,AUC_test,F1_train,F1_test
0,RandomForest,0.985831,0.925805,0.774184,0.506364
1,GradientBoosting,0.952842,0.927961,0.638310,0.532181
2,AdaBoost,0.914674,0.903883,0.439773,0.434995
3,XGBoost,0.966567,0.929573,0.690602,0.542329
4,KNN_Bagging,0.999989,0.855295,0.980458,0.246002
5,LogReg_Bagging,0.877093,0.864530,0.256042,0.260818
6,Stacking,0.999208,0.931379,0.826351,0.408751
7,Voting,0.994465,0.930711,0.843257,0.519734


In [41]:
results_df

,Model,AUC_train,AUC_test,F1_train,F1_test
0,RandomForest,0.985831,0.925805,0.774184,0.506364
1,GradientBoosting,0.952842,0.927961,0.638310,0.532181
2,AdaBoost,0.914674,0.903883,0.439773,0.434995
3,XGBoost,0.966567,0.929573,0.690602,0.542329
4,KNN_Bagging,0.999989,0.855295,0.980458,0.246002
5,LogReg_Bagging,0.877093,0.864530,0.256042,0.260818
6,Stacking,0.999208,0.931379,0.826351,0.408751
7,Voting,0.994465,0.930711,0.843257,0.519734


In [35]:
sampling_techniques = {
    'SMOTE': SMOTE(random_state=0),
    'Random Oversampling': RandomOverSampler(random_state=0),
    'Random Undersampling': RandomUnderSampler(random_state=0)
}

models = {
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'XGBoost': XGBClassifier(),
    'KNN_Bagging': BaggingClassifier(base_estimator=KNeighborsClassifier(), n_estimators=10, random_state=42),
    'LogReg_Bagging': BaggingClassifier(base_estimator=LogisticRegression(solver='liblinear'), n_estimators=10, random_state=42),
    'Stacking': StackingClassifier(classifiers=[
        RandomForestClassifier(),
        GradientBoostingClassifier(),
        AdaBoostClassifier(),
        XGBClassifier()
    ],
    meta_classifier=LogisticRegression(solver='liblinear'),
    use_probas=True, average_probas=False),
    'Voting': VotingClassifier(estimators=[
        ('RandomForest', RandomForestClassifier()),
        ('GradientBoosting', GradientBoostingClassifier()),
        ('AdaBoost', AdaBoostClassifier()),
        ('XGBoost', XGBClassifier())
    ])
}

results_list2 = []

In [38]:
for sampling_name, sampling_technique in sampling_techniques.items():
    X_train_res, y_train_res = X_train, y_train  
    
    if sampling_technique:
        X_train_res, y_train_res = sampling_technique.fit_resample(X_train, y_train)
    
    for model_name, model in models.items():
        random_search = RandomizedSearchCV(model, param_distributions=ranges[model_name],
                                           n_iter=10, scoring='roc_auc', n_jobs=-1, cv=StratifiedKFold(n_splits=3),
                                           random_state=42, verbose=2)

        random_search.fit(X_train_res, y_train_res)

        best_model = model.set_params(**random_search.best_params_)
        best_model.fit(X_train_res, y_train_res)

        y_train_pred = best_model.predict(X_train_res)
        y_test_pred = best_model.predict(X_test)

        auc_train = roc_auc_score(y_train_res, best_model.predict_proba(X_train_res)[:, 1])
        auc_test = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])

        f1_train = f1_score(y_train_res, y_train_pred)

        y_test_probs = best_model.predict_proba(X_test)[:, 1]
        thresholds = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
        best_threshold = 0
        best_f1_score_test = 0
        for threshold in thresholds:
            y_test_pred_adjusted = (y_test_probs > threshold).astype(int)
            f1_test_adjusted = f1_score(y_test, y_test_pred_adjusted)
            if f1_test_adjusted > best_f1_score_test:
                best_f1_score_test = f1_test_adjusted
                best_threshold = threshold

        f1_test = best_f1_score_test

        print('Model',model_name, 'Sampling Technique', sampling_name,
                                'AUC_train',auc_train, 'AUC_test', auc_test,
                                'F1_train', f1_train, 'F1_test', f1_test, 'Best_threshold', best_threshold,
                                'Model fitted with threshold', best_model)

        results_list2.append({
            'Model': model_name,
            'Sampling Technique': sampling_name,
            'AUC_train': auc_train,
            'AUC_test': auc_test,
            'F1_train': f1_train,
            'F1_test': f1_test,
            'Best_threshold': best_threshold,
            'Model_fitted_with_threshold': best_model
        })
        
results_df2 = pd.DataFrame(results_list2)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Model RandomForest Sampling Technique SMOTE AUC_train 0.9999999732334984 AUC_test 0.9144920712359643 F1_train 0.9992158171352298 F1_test 0.578756621131865 Best_threshold 0.4 Model fitted with threshold RandomForestClassifier(bootstrap=False, max_depth=25, min_samples_split=3,
                       n_estimators=65)
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Model GradientBoosting Sampling Technique SMOTE AUC_train 0.9991806718074291 AUC_test 0.9180284172348979 F1_train 0.9852347655666097 F1_test 0.5897291196388261 Best_threshold 0.3 Model fitted with threshold GradientBoostingClassifier(max_depth=10, min_samples_leaf=2,
                           min_samples_split=7, n_estimators=94)
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Model AdaBoost Sampling Technique SMOTE AUC_train 0.9390765291042166 AUC_test 0.8852990541455352 F1_train 0.8740563250836999 F1_test 0.5254059717129388 Best_threshold 

,Model,Sampling Technique,AUC_train,AUC_test,F1_train,F1_test,Best_threshold,Model_fitted_with_threshold
0,RandomForest,SMOTE,1.000000,0.914492,0.999216,0.578757,0.4,"(DecisionTreeClassifier(max_depth=13, max_feat..."
1,GradientBoosting,SMOTE,0.999181,0.918028,0.985235,0.589729,0.3,([DecisionTreeRegressor(criterion='friedman_ms...
2,AdaBoost,SMOTE,0.939077,0.885299,0.874056,0.525406,0.5,"(DecisionTreeClassifier(max_depth=1, random_st..."
3,XGBoost,SMOTE,0.996995,0.923349,0.970666,0.600933,0.3,"XGBClassifier(base_score=None, booster=None, c..."
4,KNN_Bagging,SMOTE,0.999993,0.862160,0.998449,0.505082,0.5,"(KNeighborsClassifier(n_neighbors=12, p=1, wei..."
5,LogReg_Bagging,SMOTE,0.913123,0.856743,0.849156,0.496129,0.6,"(LogisticRegression(C=10, penalty='l1', random..."
6,Stacking,SMOTE,0.999697,0.921392,0.989296,0.597963,0.4,StackingClassifier(classifiers=[RandomForestCl...
7,Voting,SMOTE,0.998132,0.918849,0.971349,0.596018,0.5,"VotingClassifier(estimators=[('RandomForest', ..."
8,RandomForest,Random Oversampling,1.000000,0.918013,0.999983,0.590983,0.3,"(DecisionTreeClassifier(max_depth=13, max_feat..."
9,GradientBoosting,Random Oversampling,0.998858,0.922280,0.981922,0.600000,0.5,([DecisionTreeRegressor(criterion='friedman_ms...


In [39]:
results_df2

,Model,Sampling Technique,AUC_train,AUC_test,F1_train,F1_test,Best_threshold,Model_fitted_with_threshold
0,RandomForest,SMOTE,1.000000,0.914492,0.999216,0.578757,0.4,"(DecisionTreeClassifier(max_depth=13, max_feat..."
1,GradientBoosting,SMOTE,0.999181,0.918028,0.985235,0.589729,0.3,([DecisionTreeRegressor(criterion='friedman_ms...
2,AdaBoost,SMOTE,0.939077,0.885299,0.874056,0.525406,0.5,"(DecisionTreeClassifier(max_depth=1, random_st..."
3,XGBoost,SMOTE,0.996995,0.923349,0.970666,0.600933,0.3,"XGBClassifier(base_score=None, booster=None, c..."
4,KNN_Bagging,SMOTE,0.999993,0.862160,0.998449,0.505082,0.5,"(KNeighborsClassifier(n_neighbors=12, p=1, wei..."
5,LogReg_Bagging,SMOTE,0.913123,0.856743,0.849156,0.496129,0.6,"(LogisticRegression(C=10, penalty='l1', random..."
6,Stacking,SMOTE,0.999697,0.921392,0.989296,0.597963,0.4,StackingClassifier(classifiers=[RandomForestCl...
7,Voting,SMOTE,0.998132,0.918849,0.971349,0.596018,0.5,"VotingClassifier(estimators=[('RandomForest', ..."
8,RandomForest,Random Oversampling,1.000000,0.918013,0.999983,0.590983,0.3,"(DecisionTreeClassifier(max_depth=13, max_feat..."
9,GradientBoosting,Random Oversampling,0.998858,0.922280,0.981922,0.600000,0.5,([DecisionTreeRegressor(criterion='friedman_ms...


In [48]:
results_df2_sorted = results_df2.sort_values(by='AUC_train', ascending=False)
results_df2_sorted # in descending order on the basis of best threshold

,Model,Sampling Technique,AUC_train,AUC_test,F1_train,F1_test,Best_threshold,Model_fitted_with_threshold
8,RandomForest,Random Oversampling,1.000000,0.918013,0.999983,0.590983,0.3,"(DecisionTreeClassifier(max_depth=13, max_feat..."
0,RandomForest,SMOTE,1.000000,0.914492,0.999216,0.578757,0.4,"(DecisionTreeClassifier(max_depth=13, max_feat..."
14,Stacking,Random Oversampling,0.999999,0.928408,0.995561,0.612535,0.4,StackingClassifier(classifiers=[RandomForestCl...
12,KNN_Bagging,Random Oversampling,0.999998,0.855160,0.998999,0.494516,0.5,"(KNeighborsClassifier(n_neighbors=12, p=1, wei..."
4,KNN_Bagging,SMOTE,0.999993,0.862160,0.998449,0.505082,0.5,"(KNeighborsClassifier(n_neighbors=12, p=1, wei..."
20,KNN_Bagging,Random Undersampling,0.999779,0.831379,0.994577,0.470946,0.6,"(KNeighborsClassifier(n_neighbors=12, p=1, wei..."
6,Stacking,SMOTE,0.999697,0.921392,0.989296,0.597963,0.4,StackingClassifier(classifiers=[RandomForestCl...
22,Stacking,Random Undersampling,0.999225,0.925302,0.981891,0.595976,0.7,StackingClassifier(classifiers=[RandomForestCl...
1,GradientBoosting,SMOTE,0.999181,0.918028,0.985235,0.589729,0.3,([DecisionTreeRegressor(criterion='friedman_ms...
9,GradientBoosting,Random Oversampling,0.998858,0.922280,0.981922,0.600000,0.5,([DecisionTreeRegressor(criterion='friedman_ms...


In [50]:
results_df2_sorted.loc[14]

Model                                                                   Stacking
Sampling Technique                                           Random Oversampling
AUC_train                                                               0.999999
AUC_test                                                                0.928408
F1_train                                                                0.995561
F1_test                                                                 0.612535
Best_threshold                                                               0.4
Model_fitted_with_threshold    StackingClassifier(classifiers=[RandomForestCl...
Name: 14, dtype: object

In [51]:
# The chosen model details is as displayed above